## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

response = requests.get(url, headers=headers)
# response.text

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [4]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
table = html.select("table")
len(table)

2

In [5]:
type(table)

bs4.element.ResultSet

## pandas 코드 한 줄로 데이터 수집하기

In [6]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(table))

/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/3976437143.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


In [7]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [8]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.11.01,75300.0,100.0,75400.0,76100.0,74300.0,75732.0
2,2023.10.31,75400.0,500.0,76500.0,77000.0,74800.0,79072.0
3,2023.10.30,75900.0,1100.0,74400.0,76300.0,74400.0,83365.0
4,2023.10.27,74800.0,600.0,75800.0,76300.0,74600.0,80275.0
5,2023.10.26,75400.0,1800.0,76500.0,76700.0,74100.0,98644.0
9,2023.10.25,77200.0,300.0,77300.0,78000.0,76100.0,102844.0
10,2023.10.24,76900.0,900.0,76800.0,77300.0,73500.0,129541.0
11,2023.10.23,76000.0,500.0,75500.0,76800.0,74700.0,97056.0
12,2023.10.20,75500.0,700.0,75700.0,76500.0,74200.0,125081.0
13,2023.10.19,76200.0,900.0,76200.0,77000.0,75500.0,126906.0


## 페이지별 데이터 수집 함수 만들기

In [9]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [10]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 1)

/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.11.01,75300.0,100.0,75400.0,76100.0,74300.0,75732.0
2,2023.10.31,75400.0,500.0,76500.0,77000.0,74800.0,79072.0
3,2023.10.30,75900.0,1100.0,74400.0,76300.0,74400.0,83365.0
4,2023.10.27,74800.0,600.0,75800.0,76300.0,74600.0,80275.0
5,2023.10.26,75400.0,1800.0,76500.0,76700.0,74100.0,98644.0
9,2023.10.25,77200.0,300.0,77300.0,78000.0,76100.0,102844.0
10,2023.10.24,76900.0,900.0,76800.0,77300.0,73500.0,129541.0
11,2023.10.23,76000.0,500.0,75500.0,76800.0,74700.0,97056.0
12,2023.10.20,75500.0,700.0,75700.0,76500.0,74200.0,125081.0
13,2023.10.19,76200.0,900.0,76200.0,77000.0,75500.0,126906.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [11]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(0.1)
    
    #if temp.shape[0] < 10:
    if page_no > 10:
        break;

1
2


/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


3
4


/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


5
6


/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


7
8


/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


9
10


/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))


In [12]:
len(item_list)

10

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [13]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [14]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.11.01,75300.0,100.0,75400.0,76100.0,74300.0,75732.0
2,2023.10.31,75400.0,500.0,76500.0,77000.0,74800.0,79072.0
3,2023.10.30,75900.0,1100.0,74400.0,76300.0,74400.0,83365.0
4,2023.10.27,74800.0,600.0,75800.0,76300.0,74600.0,80275.0
5,2023.10.26,75400.0,1800.0,76500.0,76700.0,74100.0,98644.0


In [15]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2023.06.13,81000.0,3500.0,77400.0,81500.0,77400.0,366343.0
10,2023.06.12,77500.0,3600.0,73900.0,78300.0,73900.0,229972.0
11,2023.06.09,73900.0,700.0,73900.0,74400.0,72800.0,91931.0
12,2023.06.08,73200.0,1300.0,74800.0,74900.0,73000.0,106918.0
13,2023.06.07,74500.0,200.0,74800.0,76900.0,74300.0,139962.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [16]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2023.11.01,75300.0,100.0,75400.0,76100.0,74300.0,75732.0,326030,SK바이오팜
2,2023.10.31,75400.0,500.0,76500.0,77000.0,74800.0,79072.0,326030,SK바이오팜
3,2023.10.30,75900.0,1100.0,74400.0,76300.0,74400.0,83365.0,326030,SK바이오팜
4,2023.10.27,74800.0,600.0,75800.0,76300.0,74600.0,80275.0,326030,SK바이오팜
5,2023.10.26,75400.0,1800.0,76500.0,76700.0,74100.0,98644.0,326030,SK바이오팜
...,...,...,...,...,...,...,...,...,...
9,2023.06.13,81000.0,3500.0,77400.0,81500.0,77400.0,366343.0,326030,SK바이오팜
10,2023.06.12,77500.0,3600.0,73900.0,78300.0,73900.0,229972.0,326030,SK바이오팜
11,2023.06.09,73900.0,700.0,73900.0,74400.0,72800.0,91931.0,326030,SK바이오팜
12,2023.06.08,73200.0,1300.0,74800.0,74900.0,73000.0,106918.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [17]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [18]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [19]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(100, 9)

## 기술통계값 구하기

In [20]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,83244.000000,1722.000000,83234.000000,84935.000000,81683.000000,208174.520000
std,6033.999963,1792.779006,5958.937265,6288.459447,5670.961919,164676.672965
min,73200.000000,0.000000,73400.000000,74000.000000,72700.000000,75732.000000
25%,79100.000000,575.000000,79100.000000,81500.000000,77275.000000,115445.750000
50%,83100.000000,1100.000000,83000.000000,84900.000000,81250.000000,150247.000000
75%,86225.000000,2125.000000,86300.000000,87925.000000,84875.000000,225306.250000
max,99700.000000,8600.000000,99800.000000,103000.000000,97500.000000,888940.000000


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [21]:
# 거래량의 과학적 기수법 읽기
1.000000e+02

100.0

In [22]:
4.701523e+05

470152.3

In [23]:
4.701523 * (10 ** 5)

470152.3

## 최근 날짜 구해서 파일명 만들기

In [24]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2023.11.01'

In [25]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2023.11.01.csv'

## 파일로 저장하기

In [26]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [27]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2023.11.01,75300.0,100.0,75400.0,76100.0,74300.0,75732.0
1,326030,SK바이오팜,2023.10.31,75400.0,500.0,76500.0,77000.0,74800.0,79072.0
2,326030,SK바이오팜,2023.10.30,75900.0,1100.0,74400.0,76300.0,74400.0,83365.0
3,326030,SK바이오팜,2023.10.27,74800.0,600.0,75800.0,76300.0,74600.0,80275.0
4,326030,SK바이오팜,2023.10.26,75400.0,1800.0,76500.0,76700.0,74100.0,98644.0
...,...,...,...,...,...,...,...,...,...
95,326030,SK바이오팜,2023.06.13,81000.0,3500.0,77400.0,81500.0,77400.0,366343.0
96,326030,SK바이오팜,2023.06.12,77500.0,3600.0,73900.0,78300.0,73900.0,229972.0
97,326030,SK바이오팜,2023.06.09,73900.0,700.0,73900.0,74400.0,72800.0,91931.0
98,326030,SK바이오팜,2023.06.08,73200.0,1300.0,74800.0,74900.0,73000.0,106918.0


## 전체 과정을 하나의 함수로 만들기

In [28]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        time.sleep(0.1)

        #if temp.shape[0] < 10:
        if page_no > 10:
            break
            
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]

    return df

In [29]:
item_code = "352820"
item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"

get_item_list(item_code, item_name)

/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))
/var/folders/cn/3xfx7dqx4zx7lmdpfl0jh9g80000gn/T/ipykernel_61727/4113515383.py:15: FutureWarning: Passing literal html to 'read_html' is de

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2023.11.01,227500.0,8000.0,220000.0,227500.0,214000.0,172591.0
2,352820,하이브,2023.10.31,219500.0,0.0,223500.0,225500.0,216000.0,130896.0
3,352820,하이브,2023.10.30,219500.0,7500.0,210000.0,222000.0,210000.0,144405.0
4,352820,하이브,2023.10.27,212000.0,8000.0,210000.0,218500.0,209000.0,312497.0
5,352820,하이브,2023.10.26,204000.0,24500.0,221500.0,222000.0,202000.0,894408.0
...,...,...,...,...,...,...,...,...,...
9,352820,하이브,2023.06.13,294000.0,0.0,294500.0,296500.0,289000.0,209505.0
10,352820,하이브,2023.06.12,294000.0,13500.0,284500.0,295000.0,281000.0,402574.0
11,352820,하이브,2023.06.09,280500.0,12500.0,270500.0,286500.0,269000.0,430504.0
12,352820,하이브,2023.06.08,268000.0,1000.0,267500.0,271000.0,265000.0,151260.0
